In [2]:
#
# Import libraries
#

import os
import pandas as pd
from dotmap import DotMap
import requests
import json
import numpy as np
from tqdm import tqdm
import time

In [3]:
modes = ['cp', 'cn']
f2i_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\OriginalFiles\f2i.xlsx"
f2i = DotMap({
    'cp': pd.read_excel(f2i_path, sheet_name='C18P').set_index('fid'),
    'cn': pd.read_excel(f2i_path, sheet_name='C18N').set_index('fid'),
})

In [4]:
#
# Obtain feature data for each CMM table and run TurboPutative
#

xm = pd.read_csv(
    r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4\WorkingFiles\Xm_norm.tsv", 
    sep='\t', index_col=0)

xm.index = [f'seqn_{i}' for i in xm.index]

In [5]:
#
# Adducts
#

posAdd = ["M+H", "M+2H", "M+Na", "M+K", "M+H-H2O", "M+H+HCOONa", "M+NH4"]
negAdd = ["M-H", "M-2H", "M+Cl", "M-H-H20", "M-H+HCOONa", "M+Na-2H"]

In [4]:
#
# Read filtered features
#

# mfilt = pd.read_csv(
#     r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\00-FeatureSelection\ALDH4\mfilt.tsv",
#     sep='\t', index_col=0
# )


# mz = f2i.cp.loc[
#     mfilt.index[np.isin(mfilt.index, f2i.cp.index)],
#     'Apex m/z'
# ].tolist()

In [6]:
f2i_filt = DotMap()
f2i_filt.cp = f2i.cp[np.isin(f2i.cp.index, xm.columns)]
f2i_filt.cn = f2i.cn[np.isin(f2i.cn.index, xm.columns)]

In [42]:
n=5

mzL = f2i_filt.cp['Apex m/z'].tolist()
mz = [mzL[i:i+n] for i in range(0, len(mzL), n)]

In [43]:
cmm = DotMap()

uri = "http://ceumass.eps.uspceu.es/mediator/api/v3/batch"

# C18P
res_all = []

for mzi in tqdm(mz):
    res = requests.post(
        uri, headers={'Content-Type': 'application/json; charset=utf-8'}, 
        json={
        "metabolites_type": "all-except-peptides",
        "databases": ["all-except-mine"],
        "masses_mode": "mz",
        "ion_mode": "positive",
        "adducts": posAdd,
        "tolerance": 10.0,
        "tolerance_mode": "ppm",
        "masses": mzi
        }
    )
    res_all.extend(res.json()['results'])
    time.sleep(2)

cmm.cp = pd.DataFrame(res_all)

100%|██████████| 446/446 [18:08<00:00,  2.44s/it]


In [44]:
# mz = f2i.cn.loc[
#     mfilt.index[np.isin(mfilt.index, f2i.cn.index)],
#     'Apex m/z'
# ].tolist()
# mz = [mz[i:i+n] for i in range(0, len(mz), n)]

n = 2 # chunk size

mzL = f2i_filt.cn['Apex m/z'].tolist()
mz = [mzL[i:i+n] for i in range(0, len(mzL), n)]

In [45]:
# C18N
res_all_N = []

for mzi in tqdm(mz):
    res = requests.post(
        uri, headers={'Content-Type': 'application/json; charset=utf-8'}, 
        json={
        "metabolites_type": "all-except-peptides",
        "databases": ["all-except-mine"],
        "masses_mode": "mz",
        "ion_mode": "negative",
        "adducts": ['all'],
        "tolerance": 10.0,
        "tolerance_mode": "ppm",
        "masses": mzi
        }
    )
    res_all_N.extend(res.json()['results'])
    time.sleep(2)

cmm.cn = pd.DataFrame(res_all_N)

100%|██████████| 355/355 [13:02<00:00,  2.21s/it]


In [46]:
cmmF = DotMap()
cmmF.cp = cmm.cp[np.isin(cmm.cp.adduct, posAdd)]
cmmF.cn = cmm.cn[np.isin(cmm.cn.adduct, negAdd)]

In [47]:
#Check

cmmF.cp

,identifier,EM,name,formula,adduct,molecular_weight,error_ppm,ionizationScore,finalScore,kegg_compound,...,hmdb_uri,lipidmaps_compound,lipidmaps_uri,metlin_compound,metlin_uri,pubchem_compound,pubchem_uri,inChiKey,pathways,cas
0,1858,760.58465,PC(16:1(9Z)/18:0),C42H82NO8P,M+H,759.577805,1,-2.0,-2.0,,...,http://www.hmdb.ca/metabolites/HMDB0008003,LMGP01011483,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59451,https://metlin.scripps.edu/metabo_info.php?mol...,52922448,https://pubchem.ncbi.nlm.nih.gov/compound/5292...,LJUDFFWSOQEQKW-QCFYSATCSA-N,[],NaN
1,1363,760.58465,PC(16:0/18:1(11Z)),C42H82NO8P,M+H,759.577805,1,-2.0,-2.0,,...,http://www.hmdb.ca/metabolites/HMDB0007971,LMGP01010576,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59420,https://metlin.scripps.edu/metabo_info.php?mol...,24778688,https://pubchem.ncbi.nlm.nih.gov/compound/2477...,AEPVLLKGEBHLEJ-KWNHIAGJSA-N,[],NaN
2,77146,760.58465,PE(15:0/22:1(13Z)),C42H82NO8P,M+H,759.577805,1,-2.0,-2.0,,...,http://www.hmdb.ca/metabolites/HMDB0008908,,,60351,https://metlin.scripps.edu/metabo_info.php?mol...,53479578,https://pubchem.ncbi.nlm.nih.gov/compound/5347...,MUWFEEOSXPAQIY-LQOXNGKESA-N,[],NaN
3,2162,760.58465,PC(20:0/14:1(9Z)),C42H82NO8P,M+H,759.577805,1,-2.0,-2.0,,...,http://www.hmdb.ca/metabolites/HMDB0008263,LMGP01011787,http://www.lipidmaps.org/data/LMSDRecord.php?L...,59709,https://metlin.scripps.edu/metabo_info.php?mol...,52923053,https://pubchem.ncbi.nlm.nih.gov/compound/5292...,HUTFJQFEXOTHFG-LLVYEJTBSA-N,[],NaN
4,163699,760.58465,PC(16:0/18:1(11Z))[U],C42H82NO8P,M+H,759.577805,1,-2.0,-2.0,,...,,,,39319,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,NaN,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138792,173361,385.15407,Ampicilloic acid,C16H21N3O5S,M+NH4,367.120190,0,-2.0,-2.0,,...,,,,611,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,NaN,[],32746-94-4
138793,153479,385.15407,"N-[2-(4-Chloro-phenyl)-acetyl]-N'-(4,7-dimethy...",C19H18ClN5O,M+NH4,367.119990,1,-2.0,-2.0,C11562,...,,,,69095,https://metlin.scripps.edu/metabo_info.php?mol...,135537831,https://pubchem.ncbi.nlm.nih.gov/compound/1355...,INZNPXHWACTNAT-UHFFFAOYSA-N,[],NaN
138794,173362,385.15407,Met Ser Met,C13H25N3O5S2,M+NH4,367.123560,9,-2.0,-2.0,,...,,,,16820,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,NaN,[],NaN
138795,173363,385.15407,Ser Met Met,C13H25N3O5S2,M+NH4,367.123560,9,-2.0,-2.0,,...,,,,20501,https://metlin.scripps.edu/metabo_info.php?mol...,0,https://pubchem.ncbi.nlm.nih.gov/compound/0,NaN,[],NaN


In [48]:
# Adapt columns to TP

for i in modes: 
    cmmF[i] = cmmF[i].rename(columns={
        'identifier': 'Identifier',
        'EM': 'Experimental mass',
        'adduct': 'Adduct',
        'error_ppm': 'mz Error (ppm)',
        'molecular_weight': 'Molecular Weight',
        'name': 'Name',
        'formula': 'Formula'
    })

In [49]:
# Identifiers to be removed
remid = [188282]

In [50]:
# Write MS_Table.tsv
for i in modes:
    cmmF[i][~np.isin(cmmF[i].Identifier, remid)].to_csv(f'CMM_{i}.tsv', sep='\t', index=False)

In [51]:
# Write Feature_Info.tsv
for i in modes:
    f2i[i].loc[
        :,#mfilt.index[np.isin(mfilt.index, f2i[i].index)],
        ['Apex m/z', 'RT [min]']
    ].join(xm.T, how='inner')\
            .reset_index(names='FeatureInfo_Name')\
                .to_csv(f'FInfo_{i}.tsv', sep='\t', index=False)

In [ ]:
# # Write Feature_Info.tsv
# for i in modes:
#     f2i[i].loc[
#         mfilt.index[np.isin(mfilt.index, f2i[i].index)],
#         ['Apex m/z', 'RT [min]']
#     ].join(xm.T, how='inner')\
#             .reset_index(names='FeatureInfo_Name')\
#                 .to_csv(f'FInfo_{i}.tsv', sep='\t', index=False)


In [32]:
xm

,P1,P2,P3,P4,P6,P7,P8,P9,P10,P11,...,N1130,N1132,N1136,N1140,N1144,N1146,N1149,N1157,N1163,N1165
seqn_LD2169,-0.414481,1.083917,-0.137611,-0.501769,-1.084914,0.691925,1.208685,0.372302,0.307655,0.229605,...,1.458137,-0.069473,1.280729,2.069009,0.355674,-0.744117,-0.076870,1.189542,-0.501087,-0.042662
seqn_LD2170,0.771123,-0.128723,0.566504,0.637105,1.879634,-1.814662,-1.516700,0.825935,1.255284,-0.919807,...,0.175212,0.485257,-0.163253,0.715568,0.348180,1.005430,0.225349,0.320045,0.264071,-0.412786
seqn_LD2171,-0.221210,-1.017524,-0.190560,0.289704,0.850356,-0.289215,-0.267745,1.208626,-0.068590,-1.011448,...,-0.038702,-1.724893,-0.234246,-1.142008,-1.175073,-1.037922,-0.423494,-0.973575,-0.588195,-0.649106
seqn_LD2243,1.181282,-1.126865,-0.264693,0.980566,0.302137,-0.747678,-0.692696,1.153231,-0.324986,-0.865972,...,-0.584945,1.480825,0.557401,-0.073520,1.328237,1.147465,0.946284,0.956701,1.573441,1.052457
seqn_LD2244,0.295432,0.897484,-2.585537,-0.102400,-1.242098,1.304763,0.016070,1.323445,-1.688522,2.511589,...,0.660388,-0.422663,1.419350,-2.155790,0.477565,-1.180798,-1.742811,0.259278,0.053532,-0.936227
seqn_LD2245,1.623541,-2.403156,-0.045565,0.168742,0.605062,-2.256618,-2.117341,0.297637,-0.227484,-1.902213,...,-1.057322,1.324112,0.990052,0.292729,1.483830,1.587641,1.090606,1.401736,1.930106,1.268888
seqn_LD2162,-0.412519,0.363480,1.341616,-0.474133,0.197589,-0.026890,-0.770295,-1.086853,1.159816,0.096871,...,0.973093,-0.976563,-0.608786,0.890970,-0.815338,-1.256123,-0.405888,-0.268514,-1.100395,-1.144571
seqn_LD2163,0.236139,0.185810,0.959677,0.551424,0.696793,0.358016,-0.012191,-0.808158,0.252678,0.283041,...,-0.068359,1.517304,-0.009516,0.924296,1.623127,1.609168,1.481778,1.370703,0.922954,0.994648
seqn_LD2164,-0.341833,1.068022,0.655048,0.079162,0.192810,0.651818,0.778785,-0.984844,1.187678,0.333531,...,-0.510137,0.528907,-1.420637,0.725317,-0.100710,0.042241,-0.652472,-0.782317,-0.162194,-1.525399
seqn_LD2213,-1.674727,1.654882,0.939744,-2.055677,-0.393829,1.960015,1.717094,-0.821737,1.631332,1.235787,...,-0.295523,1.117131,0.581896,-1.156521,0.696611,1.021714,-0.691924,0.258970,0.388612,0.595712
